In [53]:
import matplotlib
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [54]:
fisiere = [
    ("v1_world-happiness-report-2017.csv", "v1"),
    ("v2_world-happiness-report-2017.csv", "v2"),
    ("v3_world-happiness-report-2017.csv", "v3")
]

In [55]:
def regresie_simpla(x, y):
    """
    Calculează coeficienții unei regresii liniare simple (y = ax + b)
    folosind metoda celor mai mici pătrate (OLS).
    """
    n = len(x)  
    suma_x = sum(x)
    suma_y = sum(y)
    suma_x2 = sum(val ** 2 for val in x) 
    suma_xy = sum(val * yi for val, yi in zip(x, y))  

    panta = (n * suma_xy - suma_x * suma_y) / (n * suma_x2 - suma_x ** 2)
    intercept = (suma_y - panta * suma_x) / n
    return panta, intercept


In [56]:
def regresie_multipla(X, y):
    """
    Rezolvă regresia liniară multiplă folosind OLS:
    y = b0 + b1*X1 + b2*X2 + ... + bn*Xn
    """

    def transpusa(matrice):
        return [list(linie) for linie in zip(*matrice)]

    def inmultire_matrici(A, B):
        return [
            [sum(a * b for a, b in zip(linie_A, col_B)) for col_B in zip(*B)]
            for linie_A in A
        ]

    def inversa(matrice):
        n = len(matrice)
        identitate = [[float(i == j) for j in range(n)] for i in range(n)]
        aug = [linie + identitate[i] for i, linie in enumerate(matrice)]

        for col in range(n):
            pivot = max(range(col, n), key=lambda r: abs(aug[r][col]))
            aug[col], aug[pivot] = aug[pivot], aug[col]
            factor_pivot = aug[col][col]
            aug[col] = [val / factor_pivot for val in aug[col]]
            for r in range(n):
                if r != col:
                    factor = aug[r][col]
                    aug[r] = [val - factor * aug[col][i] for i, val in enumerate(aug[r])]
        return [linie[n:] for linie in aug]

    # Aplicăm formula OLS pentru regresie multiplă: (X^T * X)^-1 * (X^T * y), 
    # Returnăm coeficienții obținuți (intercept + coeficienți pentru fiecare feature).
    X_aug = [[1.0] + list(rand) for rand in X]
    XT = transpusa(X_aug)
    XTX = inmultire_matrici(XT, X_aug)
    XTX_inv = inversa(XTX)
    y_col = [[yi] for yi in y]
    XTy = inmultire_matrici(XT, y_col)
    coef = inmultire_matrici(XTX_inv, XTy)
    return [c[0] for c in coef]


In [57]:
def evalueaza_model(y_real, y_pred):
    """
    Calculează metrici de evaluare pentru un model de regresie:
    - MSE: Mean Squared Error (eroare medie pătratică)
    - R^2: coeficientul de determinare (cât de bine se potrivește modelul)
    """
    mse = mean_squared_error(y_real, y_pred)
    r2 = r2_score(y_real, y_pred)
    return mse, r2


In [58]:
def afiseaza_plot_regresie(X, y, y_pred, titlu, xlabel, ylabel, labels=("Date reale", "Regresie")):
    """
    Afișează un grafic de regresie simplă:
    - Puncte roz: date reale
    - Linie mov: predicția modelului (y_pred)
    """
    plt.figure(figsize=(8, 5))  
    plt.scatter(X, y, color="plum", label=labels[0])
    plt.plot(X, y_pred, color="lightpink", linewidth=2, label=labels[1])
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(titlu)
    plt.legend()
    plt.tight_layout()
    plt.grid(True)
    plt.show()

In [59]:
def afiseaza_plot_regresie_multipla(X, y, y_pred, xlabel, titlu):
    """
    Afișează două grafice de tip scatter:
    - pentru fiecare predictor (GDP și Freedom) în raport cu Happiness
    - suprapune predicțiile modelului peste datele reale
    """
    plt.figure(figsize=(12, 5))  

    plt.subplot(1, 2, 1)
    plt.scatter(X[:, 0], y, color="plum", label="Date reale")  
    plt.scatter(X[:, 0], y_pred, color="lightpink", marker="x", label="Predicții")
    plt.xlabel("Economy..GDP.per.Capita.")
    plt.ylabel("Happiness.Score")
    plt.title(f"{titlu}: GDP vs Happiness")
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.scatter(X[:, 1], y, color="plum", label="Date reale") 
    plt.scatter(X[:, 1], y_pred, color="lightpink", marker="x", label="Predicții")
    plt.xlabel("Freedom")
    plt.ylabel("Happiness.Score")
    plt.title(f"{titlu}: Freedom vs Happiness")
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

In [60]:
def regresie_gdp(df, eticheta):
    """
    Aplică regresie liniară simplă pentru a estima scorul de fericire în funcție de GDP.
    Folosește atât un model din sklearn, cât și o implementare manuală cu metoda celor mai mici pătrate.
    Afișează coeficienții, metricile de performanță și graficul comparativ.
    """
    print("\n Regresie pe GDP")
    y = df["Happiness.Score"].values 
    X_gdp = df[["Economy..GDP.per.Capita."]].values 

    # Model automat (sklearn)
    model = LinearRegression().fit(X_gdp, y)
    y_pred = model.predict(X_gdp)
    mse, r2 = evalueaza_model(y, y_pred)
    print(f"[Tool] GDP -> Panta: {model.coef_[0]:.4f}, Intercept: {model.intercept_:.4f}, MSE: {mse:.4f}, R2: {r2:.4f}")

    # Model manual 
    x_flat = X_gdp.flatten()
    panta, inter = regresie_simpla(x_flat, y)
    y_pred_m = [panta * val + inter for val in x_flat]
    mse_m, r2_m = evalueaza_model(y, y_pred_m)
    print(f"[Manual] GDP -> Panta: {panta:.4f}, Intercept: {inter:.4f}, MSE: {mse_m:.4f}, R2: {r2_m:.4f}")

    afiseaza_plot_regresie(X_gdp, y, y_pred,
                       f"{eticheta} - Happiness vs GDP",
                       "GDP per Capita", "Happiness Score")


In [61]:
def regresie_family(df, eticheta):
    """
    Aplică regresie liniară simplă pentru a estima fericirea în funcție de scorul Family.
    Se folosesc ambele metode (tool și manual).
    Se evaluează performanța și se afișează graficul rezultat.
    """
    print("\n Regresie pe Family")
    y = df["Happiness.Score"].values
    X_family = df[["Family"]].values

    # Model cu sklearn
    model = LinearRegression().fit(X_family, y)
    y_pred = model.predict(X_family)
    mse, r2 = evalueaza_model(y, y_pred)
    print(f"[Tool] Family -> Panta: {model.coef_[0]:.4f}, Intercept: {model.intercept_:.4f}, MSE: {mse:.4f}, R2: {r2:.4f}")

    # Varianta manuală
    x_flat = X_family.flatten()
    panta, inter = regresie_simpla(x_flat, y)
    y_pred_m = [panta * val + inter for val in x_flat]
    mse_m, r2_m = evalueaza_model(y, y_pred_m)
    print(f"[Manual] Family -> Panta: {panta:.4f}, Intercept: {inter:.4f}, MSE: {mse_m:.4f}, R2: {r2_m:.4f}")

    afiseaza_plot_regresie(X_family, y, y_pred,
                           f"{eticheta} - Happiness vs Family",
                           "Family", "Happiness Score")


In [62]:
def regresie_multipla_gdp_freedom(df, eticheta):
    """
    Aplică regresie multiplă pentru a estima fericirea în funcție de GDP și Freedom.
    Se folosește atât modelul automat, cât și implementarea manuală OLS.
    Se compară coeficienții și se afișează două grafice (GDP și Freedom vs Happiness).
    """
    print("\n Regresie multiplă pe GDP și Freedom")
    y = df["Happiness.Score"].values
    X = df[["Economy..GDP.per.Capita.", "Freedom"]].values

    # Model sklearn
    model = LinearRegression().fit(X, y)
    y_pred = model.predict(X)
    mse, r2 = evalueaza_model(y, y_pred)
    print(f"[Tool] Multi -> Intercept: {model.intercept_:.4f}, Coef: {model.coef_}, MSE: {mse:.4f}, R2: {r2:.4f}")

    # Model manual
    coef = regresie_multipla(X, y)
    y_pred_m = [coef[0] + coef[1] * x[0] + coef[2] * x[1] for x in X]
    mse_m, r2_m = evalueaza_model(y, y_pred_m)
    print(f"[Manual] Multi -> Intercept: {coef[0]:.4f}, Coef GDP: {coef[1]:.4f}, Coef Freedom: {coef[2]:.4f}, MSE: {mse_m:.4f}, R2: {r2_m:.4f}")

    afiseaza_plot_regresie_multipla(X, y, y_pred, y_pred_m, eticheta)


In [63]:
def proceseaza_fisier(cale, eticheta):
    """
    Încarcă datele din fișier și rulează toate cele 3 tipuri de regresii.
    """
    print(f"\n Procesare fișier: {eticheta}")
    df = pd.read_csv(cale).dropna()
    
    regresie_gdp(df, eticheta)
    regresie_family(df, eticheta)
    regresie_multipla_gdp_freedom(df, eticheta)


In [ ]:
print("Panta (coef) = cât de mult influențează acea variabilă scorul fericirii (dacă e pozitivă → influență pozitivă).");
print("Intercept = valoarea fericirii prezisă când toate variabilele sunt 0 (doar o constantă, utilă pt. model, dar nu interpretabilă direct).");
print("MSE (Mean Squared Error) = cât de „departe” sunt predicțiile modelului față de valorile reale (cu cât mai mic, cu atât mai bine).");
print("R² (R squared) = cât de bine explică modelul variația din date. Variază între 0 și 1. Aproape de 1 → model bun.");

for cale, eticheta in fisiere:
    proceseaza_fisier(cale, eticheta)